# Setting of stack phase 

At this phase we are going to set the stacked-phase dataset. This method is based just in apply method. It implies less memory.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
from utils.trainFold import getVector

In [3]:
import sys
sys.path.append('../dsbase/src/main')
from AdaBoostClassificationDSBase import AdaBoostClassificationDSBaseModel

## Loading the original stacked dataset and shuffle it

In [4]:
#df = pd.read_csv('datasets/train_stack.csv')

In [5]:
#df_frac = df.sample(frac=1)

In [6]:
#df_frac.to_csv('datasets/train_stack_shuffled.csv')

In [7]:
# cat datasets/train_stack_shuffled.csv | head -n +4500001 > datasets/train_stack_shuffled_reduced.csv

In [8]:
df = pd.read_csv('datasets/train_stack_shuffled_reduced.csv')

In [9]:
df.shape

(450000, 78)

## Defining the Fold X processing 

### Defining helping functions 

In [10]:
def loadColumnCategoricalOrder(df, columns_categorical):
    columns_categorical_order_dict = {}
    for x in columns_categorical:
        columns_categorical_order_dict[x] = np.where(df.columns == x)[0][0]
    return columns_categorical_order_dict

In [11]:
def loadColumnCategoricalVectors(fold_id, columns_categorical):
    columns_categorical_vectors_dict = {}
    out_path = 'models/fold' + str(fold_id)
    for c in columns_categorical:
        vec = np.load('models/fold' + str(fold_id) + "/" + c + ".sav.npy")
        columns_categorical_vectors_dict[c] = vec
    return columns_categorical_vectors_dict

In [12]:
def loadModel(fold_id):
   # --------------------------------------
    # Load the i-th model and process
    print('   loading model ...')    
    model = AdaBoostClassificationDSBaseModel('AB2',None,None,None,None,None,None)
    model.load('models/fold' + str(fold_id))
    return model

In [13]:
def calculateF1(x, cc, cc_o, cc_v, model):
    xnp = x.values
    acc=0
    for c in cc:
        index = cc_o[c] + acc
        vec = cc_v[c]
        new = getVector(xnp[index], vec)
        xnp = np.delete(xnp, index)
        xnp = np.insert(xnp, index, new)
        acc += (new.size - 1)
    pre_result = model.scalerX.transform(xnp.reshape(1,-1))
    result = model.model.predict_proba(pre_result)
    return result[0,1]

### calculating support variables

In [14]:
df_w = df.drop(['MachineIdentifier','HasDetections','fold'], axis=1)
columns_categorical = df_w.select_dtypes(include=['object']).columns

In [15]:
cc_order = loadColumnCategoricalOrder(df_w,columns_categorical)

In [16]:
N = 9
for i in range(1,N+1):
    print('-------- Process Fold ',i,' -------------------')
    print('loading vectors ...')
    cc_values_f = loadColumnCategoricalVectors(i,columns_categorical)
    print('loading model ...')
    model_f = loadModel(i)
    print('applying folding prediction ...')
    df['f' + str(i)] = df_w.apply(func=calculateF1, axis=1, args=(columns_categorical, cc_order, cc_values_f, model_f))
    # save security DatFrame
    df['f' + str(i)].to_csv('datasets/f_stack.csv.' + str(i))

-------- Process Fold  1  -------------------
loading vectors ...
loading model ...
   loading model ...
initiating empty model AB2. AdaBoostClassification
loading model: models/fold1/AdaBoostClassification_AB2.sav
applying folding prediction ...


/anaconda3/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.20.2 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator AdaBoostClassifier from version 0.20.2 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.20.2 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


ValueError: ('operands could not be broadcast together with shapes (1,6862) (6861,) (1,6862) ', 'occurred at index 0')

## Lets obtain the final stacked dataset 

In [ ]:
df[['HasDetections','fold','f1','f2','f3','f4','f5','f6','f7','f8','f9']].describe()

In [ ]:
df.to_csv('datasets/train_stack_set.csv')

# End of stack train setting!! 

Local Environment: stimated time -> 322 sec / 1000 elements
AWS EC2:  stimated time -> 322 sec / ???elements